# [Youtube](https://www.youtube.com/watch?v=tkwZ1jG3XgA) | [Slides](https://www.youtube.com/watch?v=tkwZ1jG3XgA) | Date: March 8 2020

# Database Backends

- This is the boundary, the lowest you can go in terms of Database integration in Django area. Beyond this you are no longer in Django. Beyond this, you enter your database driver
- Made of few components/classes
    - **DatabaseWrapper**
        - Basic info about databases
        - DB Column type to Django field type mappings
        - Somewhat about transaction behavior
    - **DatabaseOperations**
        - Flushing, sequence resets, typecasting, etc.
    - **DatabaseFeatures**
        - What your DB supports
        - What all type exists in your DB
        - How null comparison is handled
    - **DatabaseCreation**
        - old syncdb command
        - mostly there to let us know how to create certain types of things, specially indexes
    - **DatabaseIntrospection**
        - `inspectdb` command
            - Neat feature in django, django can look at your db structure and reverse engineer that into models file using `inspectdb`
        - Even more getting into the guts of database driver
        - Some interesting things to learn there specially if you look at MySQL
    - **DatabaseSchemaEditor**
        - Introduced in 1.7
        - This is how migrations are done
        - Generate bunch of sql alter statements based on what database engine you are using with your Django app
    - **DatabaseClient**
        - `dbshell` management command
            - Basically opens up an interact prompt directly into your database
    - **Should you write a database backend?**
        - Probably not!
        - If we don't support driver of your choice only then think about this.
    - **SQLCompiler**
        - Turn Django query instance into SQL for your database
        - Subclasses for non-SELECT queries, such as `SQLInsertCompiler` for `INSERT`, `SQLDeleteCompiler` for `DELETE`, etc.

        - The Bridge
            - This is the bridge between higher level queries you work with and the low level database
            - `Query.get_compiler()` returns a `SQLCompiler` instance for that Query.
            - Which calls the `compiler()` method of `DatabaseOperations`, passing the name of the desired compiler
            - Which in turn looks at `DatabaseOperations.compiler_module` to find location of `SQLCompiler` classes for current backend
            - **Here be Dragons**
                - `SQLCompiler` - scary, complex code
                - `as_sql()` method is where the ugly magic happens.
                - Pokes into attribs of the Query to find the building blocks, and `DatabaseOperations` to help translate to appropriate SQL.
        - MySQL and Oracle provide custom compilers
            - Probably something you never wanna write your own version of
        - MySQL needs it because it provides subqueries
    - **Query**
        - We are in the guts of Django now, good to fiddle with here
        - Basic `Query` and `RawQuery` (for `raw()`)
            - django/db/models/sql
        - Basic structure
            - Attribs representing all parts of it
            - Attribs for table, select clause, where clause, etc.
            - `as_sql()` pulls all of this together, works with SQL Compiler and makes a nice beautiful SQL query out of it
        - A Query is a tree, uses tree like data structure
            - Turning it into SQL is iterating over all of its parts, each of which has a `as_sql()` method, concatenate results and you see your SQL Query.
        - Django does this pattern a lot (iterating and building things, we will also see this in template structure)
        - **Here be dragons**
            - Over 2k lines of code in django/db/models/sql/query.py and that's not all of it, there are more files to it in django/db/models/sql
            - **WHY?**
                - Reason for this complexity is that all higher level constructs like `filter`, etc are all infinitely, arbitrarily chainable
                - You can take that, call `filter`, get back a QuerySet, call `filter` again or a `select_related` or something else on a QuerySet and go on chaining it further
                    - Basically we have to see how to do anything you are allowed to do in Django, chain it on to anything else you are allowed to do in Django, merge it all together into something that actually makes sense - This is difficult to do and requires a lot of code.
                    - Most of the complexity of `Query` comes from the merging process. _Every_ legal combination of Query methods and option has to be supported here, plus hooks for custom extensions.
                    - Actual bits that generate SQL are very small, most of it is bookkeeping - maintaining records, etc.
                    - Most of the times it doesn't modify anything in place, it clones and builds a completely new instance.
                        - As a result, you almost never want to write a custom subclass of `Query`, instead subclass `QuerySet` and work at a higher level.
                        - Except...
        - **Query Expressions and Query Functions**
            - Now build into Django - first class support
            - Expressions - Allows complex logic and references, access to more advanced SQL constructs
                - E.g. you got an integer field, you want to select a bunch of instances of it and increment by 1, used to be no way to do this in Django, but now that we have Query Expressions, Query functions, you can do this.
            - Built-In examples:
                - `When` - implements `WHEN ... THEN` in SQL
                - `Case` - implements `CASE` statements in SQL
                - `F` - implements references to columns or annotations
                - `Func` implements calls to SQL functions
            - **Database Functions**
                - Implemented using `Func` query expression
                - Ability to call SQL functions
                - Built-in examples - support for `UPPER`, `SUBSTRING`, `COALESCE`
                - Basically anything that your database support as a SQL level function, you can wrap it around into one of these and start using in the ORM
        - **Custom Lookups**
            - Added in Django 1.7
            - subclass `django.db.models.Lookup`, implement `as_sql()` and give it a lookup_name
            - Can also transform results by subclassing `Transform` intead of `Lookup`
    - **QuerySet**
        - Begin getting into terminology that the tutorial uses
        - Wraps a `Query` instance into a nice API
        - `django/db/models/query.py`
        - **Laziness** by default most of the times
            - **How to force execution of a queryset?**
                - Call method that doesn't return QuerySet
                - Call method that requires actual database querying
                - Including special Python methods
        - **Custom Class Behaviour**
            - This is Python Specific - See appropriate tutorial about [Python class developers toolkit - Raymond Hettinger](https://www.youtube.com/watch?v=HTLu2DFOdTg)
            - Basically in Python you can write a class that emulates all sorts of behavior that Python supports, example, you can write a class thats iterable by implementing `__iter__` method.
            - So that's what allows QuerySet to be used in a for loop or list comprehension, etc.
            - Other special methods implemented by `QuerySet` class include `__len__`, `__bool__`, `__getitem__`, `__nonzero__`
            - One special thing done in QuerySet class - `__repr__` will perform a Query and will add `LIMIT 21` to it
            - Saves you from `repr()`-ing a `QuerySet` with a million results in it. When Django tries to generate a string representation of it, it takes up a lot of memory
        - **It is a container**
            - Each `QuerySet` instance has an internal results cache
            - Iterating will pull results from cache if populated rather than re-executing the query
            ```
            >>>  my_queryset  =  SomeModel.objects.all()
            >>>  my_queryset[5].some_attribute  3
            >>>  my_queryset[5].some_attribute  =  2
            >>>  my_queryset[5].save()
            >>>  my_queryset[5].some_attribute  3  
            
            >>>  #  WAT
            ```
            - A performance trade-off
            - Calling `QuerySet` method will usually clone the pre-existing `QuerySet`, apply the change, and return the new instance (doing a new query)
            - Except for iteration, length/existence checks, which can re-use the existing `QuerySet` instance results cache without doing new query
        - **Useful methods**
            - `update()` - Single SQL `UPDATE` query howver if there's a custom `save` method on the model, that will not be called - hence doesn't trigger `pre_save` and `post_save` signals.
            - `delete()` - Delete a bunch of objects - Attempt to issue a single SQL `DELETE` query - cascades if you have ForeignKey relationships across ForeignKey - doesn't execute custom `delete()` method, but **does send** the `pre_delete` and `post_delete` signals _(including for things deleted by cascade)_
            - `exists()` - `if <queryset>` can be slower than `exists()` a lot of times - It is very fast special case because Django knows that this method is only used for that one purpose. Almost always faster than any other check.
            - `defer()` / `only()` - Let you do query on partial models, example - "Hey, fetch every field on that model, **except these**" and `only` means - fetch **only** field fields. Lets you have fine grained control.
                - Access to unqueried fields will result in a new query to fetch the data.
            - `values()` / `values_list()` - These methods return a subclass of QuerySet. Let you control what fields come back but don't give you model instances.
                - `values` - returns dictionary - keys are names of the fields
                - `values_list` - returns lists
                    - `values_list(flat=True)` returns a single flattened list
            - **Solving the N+1 Problem**
                - `select_related` - get results  plus foreign-key related objects in a **single** query (joining in SQL)
                    - **Limitation:** - Only works with foreign key and 1:1 relations 
                - `prefetch_related` - can fetch many-to-many and generic relations with one query per relation (joining in Python)
            - **Some other fun methods:**
                - `update_or_create` - like `get_or_create` but lokos for an existing instance to update
                - `select_for_update` - locks the selected rows until end of transaction
                - `extra` - close to `raw` but lets you add custom clauses to a regular `QuerySet`
                - See **documentation for Django QuerySets** for more.
    - **Managers**
        - The high level interface we actually work with
        - Attached to model class using self.model
        - Gives you an easier interface to a lot of methods (think QuerySet methods)
        - **OPTIONS**
            - Don't specify one at all - Django will create a default one and name it `objects`
            - If you do specify however, Django doesn't create the default `objects` manager.
            - A model can have multiple manager
            - First one defined becomes the default manager (access via `_default_manager`)
        - **HOW IT WORKS**
            - Manager's `get_queryset()` returns QuerySet for a model
            - Almost everything else just proxies through to that
            - Exception: `raw()` - it instantiates a `RawQuerySet` - It uses `RawQuery` instead of `Query`
        - Examples of what these are good for:
        ```
        from django.db import models
        
        class Entry(models.Model):
            LIVE_STATUS = 1
            DRAFT_STATUS = 2
            STATUS_CHOICES = (
                (LIVE_STATUS, 'Live'),
                (DRAFT_STATUS, 'Draft'),
            )
            status = models.IntegerField(
                choices=STATUS_CHOICES,
                default=LIVE_STATUS
            )
        ```
        - We can write a custom manager for the above that only returns live entries. Here it is:
        ```
        class LiveEntryManager(models.Manager):
            def live(self):
                return self.filter(
                    status=self.model.LIVE_STATUS
                )
        
        class Entry(models.Model):
            ...
            objects = LiveEntryManager()
            
        live_entries = Entry.objects.live()
        ```
        - Very basic, but James instead recommends writing a custom queryset, example:
        ```
        class EntryQuerySet(models.QuerySet):
            def live(self):
                return self.filter(
                    status=self.model.LIVE_STATUS
                )
        
        class EntryManager(models.Manager):
            def get_queryset(self):
                return EntryQuerySet(self.model)

        live_in_april = Entry.objects.filter(
            pub_date__year=2015,
            pub_date__month=4
        ).live()
        ```
        - Generally better than writing a custom manager - first write a custom QuerySet and then write a custom Manager on that custom QuerySet.
        - **CAUTION**
            - Overriding `get_queryset()` will affect _all_ queries made through that manager.
            - Usually best to keep a vanilla manager around so you can access everything normally.
    - **MODELS**
        - Actual representation of the data and the associated logic
        - **The MODEL METACLASS*
            - `django.db.models.base.ModelBase`
            - Does the basic setup of the model class
            - Handles `Meta` declaration, sets up the default manager if needed, adds in model-specific exception (e.g. for model `Entry`, there is `Entry.DoesNotExist` subclasses, etc.
            - Talking about meta classes in Python - since that's what Django uses here
                - `__init__` is not a constructor, it is an initializer
                - `__new__` is the constructor and it comes from a baseclass called `type` 
                - A `metaclass` is a subclass of `type` which overrides `__new__` to control how classes get created.
            - Way this works in Django
                - You write your model class and you nust define a field or two and lots of things show up that you didn't write.
                - Eg. if you give a `date` field, it will suddently have methods called `get_next_by_<field_name>()` and `get_previous_by_<field_name>`
                - Answer here is - Django uses metaclass to reach in and tinker our class definition as Python is defining it (as the code gets executed)
            - Most of the actual heavylifting is __not__ done in metaclass itself; instead, anything which requires special behaviour - like many model fields - should define a method named `contribute_to_class()`, which will be called and passed the model class and the name it will be bound to in the class.
            - `ModelBase` then loops through the attribute dictionary of the new class, and calls `contribute_to_class()` on anything which defines it.
        - **MODEL FIELDS**
            - subclasses of `django.db.models.Field`
            - **DATA TYPES**
                - Fields do a lot of heavy lifting, one of the thing they care about is data types
                - Every field can define these methods:
                    - `get_internal_type()` - return a built-in field type if similar enough to that type
                    - Or `db_type()` - return a custom database-level type for the field.
            - **VALUE CONVERSION**
                - Model fields handle the value conversion
                - `to_python()` converts from DB-level type to correct Python type
                - `value_to_string()` - converts to string for serialization purpose
                - Multiple methods for preparing values for various kinds of DB-level use (querying, storage, etc.)
                - Anytime you are quering for data for sending data to your database, we have to convert - we use these methods to convert.
            - **Some other USEFUL METHODS**
                - Every field has a method called `formfield()` - returns a default form field to use for this field - this is what Django ModelForms use
                - `value_from_object()` - Given an instance of a model, tells you what value that field has on that model
                - `deconstruct()` - Used for data migration  
                    - Schema migrations change the database, while data migrations change the data
                    - E.g. like if you are creating a new column and you want to populate it from something else, you'd do a data migration to do it. 
                    - this method is the reverse of `__init__` - this method is giving us something that if you fed into `__init__` would be correct.
        - **MODEL INHERITANCE**
            - For a long time it didn't use to work
            - Now we have multiple types of inheritance
                - Abstract parents
                - Multi-table
                - Proxy models
            - Abstract parents
                - They have `abstract = True` in the `Meta` declaration
                - Dont' create a database table and cannot be directly instantiated because it is an abstract class.
                - Subclasses, if not abstract, generate a table with both their own fields and the abstract parent's.
                - Subclasses can subclass and/or override a parent's `Meta`.
                - Big example of where it is used: Django's Auth system - there is a class called `AbstractUser`
                    - `AbstractUser` - It defines the bare min API that must be supported that Django's auth system as use as a representation of User account
                    - Built in django `User` model is a subclass of it but we can write our own subclass if needed.
            - Multi-table Inheritance
                - No special syntax - just subclass the other model
                - Can't directly subclass parent `Meta`, but can override by re-declaring options.
                - Can add new fields, managers, etc.
                - Subclass has implicit `OneToOneField` to parent
            - Proxy Models
                - Can be created by declaring `proxy = True` in `Meta`
                - Will reuse parent's table and fields; only allowed python level changes - never the database level
                - You can define additional methods, managers, etc.
                - Queries against the proxy models return instances of proxy not instances of the parent
                - Think about it usefullness as: Have you ever had an app that did most of what you wanted except you wanted this one extra method?
            - Unmanaged Models
                - Related concept as of Proxy Models
                - Set `managed = False` in `Meta`
                - Django won't do any DB Management for this model: won't create tables, won't track migrations, won't auto-add a primary key if missing, etc.
                - Can declare fields, define `Meta` options, define methods normally.
                - Its USage:
                    - Unmanaged models are mostly useful for wrapping a pre-existing DB table, or DB view, that you don't want Django to try to mess with.
                    - While they can be used to emulate some aspects of model inheritance, declaring a proxy subclass of a model is almost always a better way of doing that.

# THE FORMS LIBRARY

## Major Components

- Forms
- Fields
- Widgets
- Model <-> form conversion
- Media Support

### Widgets _(We start at the lowest level with widgets)_
**Note: Here onwards - Mostly information will be on slides, additionally anything said during video, if interesting, or hacky _so being aware of it is important_ will be added to the notes here in notebook**

- Each widget has `value_from_datadict` and `render` methods - `value_from_datadict` - gets the value submitted during form submission, `render` is responsible for generating the widget's HTML given a type of data
- MultiWidgets are special case - mainly used for _datetime_ field to split date and time.
- Fields - every field has a widget associated with it to render.

